In [19]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk

In [20]:
food_data = pd.read_csv("food-dataset-extended-2.csv", encoding="utf-8")

In [21]:
rating_columns = ["spice level", "sweet level", "sour level", "salt level", "bitter level"]
food_data[rating_columns] = food_data[rating_columns] / 10.0

ratings = food_data[rating_columns].mean(axis=1).values
dishes = np.arange(len(food_data)) 

X_train, X_test, y_train, y_test = train_test_split(dishes, ratings, test_size=0.2, random_state=42)

dish_input = Input(shape=(1,), name="dish_input")
dish_embedding = Embedding(input_dim=len(food_data), output_dim=16, name="dish_embedding")(dish_input)
dish_embedding = Flatten()(dish_embedding)

x = Dense(64, activation='relu')(dish_embedding)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(16, activation='relu')(x)

output = Dense(1, activation='sigmoid', name="output_layer")(x)

ncf_model = Model(inputs=dish_input, outputs=output)
ncf_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
ncf_model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0261 - mae: 0.1319 - val_loss: 0.0222 - val_mae: 0.1214
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0217 - mae: 0.1197 - val_loss: 0.0219 - val_mae: 0.1196
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0214 - mae: 0.1151 - val_loss: 0.0221 - val_mae: 0.1203
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0170 - mae: 0.1018 - val_loss: 0.0228 - val_mae: 0.1218
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0177 - mae: 0.0989 - val_loss: 0.0241 - val_mae: 0.1249
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0125 - mae: 0.0802 - val_loss: 0.0260 - val_mae: 0.1292
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0112 - mae: 0.0724 - val_loss: 0.0273 - val_mae: 0.1327
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0087 - mae: 0.0620 - val_loss: 0.0281 - val_mae: 0.1365
Epoch 9/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0097 - mae:

In [22]:
def recommend_dishes():
    cuisine = cuisine_var.get().strip().lower()
    taste = taste_var.get().strip().lower()
    temperature = temperature_var.get().strip().lower()
    spice_level = spice_var.get()
    sweet_level = sweet_var.get()
    salt_level = salt_var.get()
    sour_level = sour_var.get()
    bitter_level = bitter_var.get()
    
    if not cuisine and not taste and not temperature and spice_level == 0 and sweet_level == 0 and salt_level == 0 and sour_level == 0 and bitter_level == 0:
        result_label.config(text="Please provide at least one preference for better recommendations!")
        return

    user_prefs = {
        "cuisine": cuisine,
        "taste": taste,
        "temperature": temperature,
        "ratings": np.array([spice_level, sweet_level, salt_level, sour_level, bitter_level]).mean() / 10.0
    }
    
    filtered_data = food_data[(food_data['Cuisine'].str.lower() == user_prefs['cuisine']) &
                              (food_data['taste'].str.lower() == user_prefs['taste']) &
                              (food_data['temperature'].str.lower() == user_prefs['temperature'])]
    
    if filtered_data.empty:
        result_label.config(text="No exact matches found. Showing top-rated dishes instead.")
        filtered_data = food_data

    dish_indices = np.arange(len(filtered_data))
    predicted_scores = ncf_model.predict(dish_indices)
    top_n_indices = np.argsort(predicted_scores.flatten())[::-1][:5]
    
    recommended_dishes = "\n".join(filtered_data.iloc[top_n_indices]['Dish name'].tolist())
    result_label.config(text=f"Top 5 Recommended Dishes:\n{recommended_dishes}")


In [23]:



root = tk.Tk()
root.title("Food Recommendation System")

cuisine_var = tk.StringVar()
taste_var = tk.StringVar()
temperature_var = tk.StringVar()
spice_var = tk.IntVar()
sweet_var = tk.IntVar()
salt_var = tk.IntVar()
sour_var = tk.IntVar()
bitter_var = tk.IntVar()

cuisines = ["Indian", "Chinese", "American","Korean","Italian"]
tastes = ["Sweet", "Sour", "Bitter","Spicy","Salty"]
temperatures = ["Hot", "Cold"]

tk.Label(root, text="Cuisine:").pack()
ttk.Combobox(root, textvariable=cuisine_var, values=cuisines).pack()

tk.Label(root, text="Taste:").pack()
ttk.Combobox(root, textvariable=taste_var, values=tastes).pack()

tk.Label(root, text="Temperature:").pack()
ttk.Combobox(root, textvariable=temperature_var, values=temperatures).pack()

tk.Label(root, text="Spice Level (0-10):").pack()
tk.Scale(root, from_=0, to=10, orient=tk.HORIZONTAL, variable=spice_var).pack()

tk.Label(root, text="Sweet Level (0-10):").pack()
tk.Scale(root, from_=0, to=10, orient=tk.HORIZONTAL, variable=sweet_var).pack()

tk.Label(root, text="Salt Level (0-10):").pack()
tk.Scale(root, from_=0, to=10, orient=tk.HORIZONTAL, variable=salt_var).pack()

tk.Label(root, text="Sour Level (0-10):").pack()
tk.Scale(root, from_=0, to=10, orient=tk.HORIZONTAL, variable=sour_var).pack()

tk.Label(root, text="Bitter Level (0-10):").pack()
tk.Scale(root, from_=0, to=10, orient=tk.HORIZONTAL, variable=bitter_var).pack()

tk.Button(root, text="Get Recommendations", command=recommend_dishes).pack()

result_label = tk.Label(root, text="")
result_label.pack()
root.geometry("500x600") 
root.mainloop()


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
